In [1]:
using Yao
using Yao.Blocks: CompositeBlock, AbstractBlock
import Yao.Blocks:apply!

In [2]:
reg = rand_state(10)
block = kron(4, 2=>X)
c = Concentrator{10}(block, [1,3,9,2]);
apply!(copy(reg), c) == apply!(copy(reg), kron(10, 3=>X))

LoadError: [91mUndefVarError: Concentrator not defined[39m

In [3]:
reg0 = rand_state(5,3)
reg = focus!(copy(reg0), 2:3)

hcat([sum(abs2.(reshape(reg.state[i,:], :, 3)), 1)[1,:] for i in 1:4]...)'

4×3 Array{Float64,2}:
 0.248823   0.290744  0.252602
 0.435655   0.187778  0.156927
 0.235017   0.182767  0.160819
 0.0805048  0.338712  0.429653

In [4]:
reg|>probs

4×3 Array{Float64,2}:
 0.248823   0.290744  0.252602
 0.435655   0.187778  0.156927
 0.235017   0.182767  0.160819
 0.0805048  0.338712  0.429653

In [35]:
####################
# Measure Functions
####################
using StatsBase
_measure(pl::Vector, ntimes::Int) = sample(0:length(pl)-1, Weights(pl), ntimes)
function _measure(pl::Matrix, ntimes::Int)
    B = size(pl, 1)
    res = Matrix{Int}(ntimes, B)
    @simd for ib=1:B
        @inbounds res[:,ib] = _measure(pl[:,ib], ntimes)
    end
    res
end

measure(reg::AbstractRegister, nshot::Int=1) = _measure(reg |> probs, nshot)

function measure_remove!(reg::AbstractRegister{B}) where B
    state = reshape(reg.state, size(reg.state,1),:,B)
    nstate = similar(reg.state, 1<<nremain(reg), B)
    pl = reg |> probs
    res = Vector{Int}(B)
    @simd for ib = 1:B
        @inbounds ires = _measure(pl[:, ib], 1)[]
        @inbounds nstate[:,ib] = view(state, ires+1,:,ib)./sqrt(pl[ires+1, ib])
        @inbounds res[ib] = ires
    end
    reg.state = reshape(nstate,1,:)
    reg, res
end

function measure!(reg::AbstractRegister{B}) where B
    state = reshape(reg.state, size(reg.state,1),:,B)
    nstate = zero(state)
    nreg, res = measure_remove!(reg)
    _nstate = reshape(nreg.state, :, B)
    @simd for ib in 1:B
        @inbounds nstate[res[ib]+1, :, ib] = view(_nstate,:,ib)
    end
    reg.state = reshape(nstate, size(nstate, 1), :)
    reg, res
end

measure! (generic function with 1 method)

In [51]:
reg2 = copy(reg)
pre = nothing
for i in 1:5
    reg2, res = measure!(reg2)
    @test reg2 |> isnormalized
    if pre!=nothing
        println(pre, res)
        @test pre == res
    end
    pre = res
end

[31, 0, 28][31, 0, 28]
[31, 0, 28][31, 0, 28]
[31, 0, 28][31, 0, 28]
[31, 0, 28][31, 0, 28]


In [7]:
using BenchmarkTools
#@benchmark measure2!(reg2)

In [8]:
using Compat.Test
reg = rand_state(5, 3)
@test reg |> probs ≈ abs2.(reg.state)

Test Passed


In [31]:
#####################
# Measurement Blocks
#####################
abstract type AbstractMeasure <: AbstractBlock end
import Yao:nqubits
import Yao.Blocks:AnySize, datatype
nqubits(::Type{T}) where {T <: AbstractMeasure} = AnySize
nqubits(m::AbstractMeasure) = AnySize
datatype(m::AbstractMeasure) = Bool
# TODO: add WorkerPool to this block to specify workers for parallelled sampling
mutable struct Measure <: AbstractMeasure
    result::Vector{Int}
    Measure() = new()
end

function apply!(reg::AbstractRegister, block::Measure)
    _, samples = measure!(reg)
    block.result = samples
    reg
end

mutable struct MeasureAndRemove <: AbstractMeasure
    result::Vector{Int}
    MeasureAndRemove() = new()
end

function apply!(reg::AbstractRegister, block::MeasureAndRemove)
    reg, samples = measure_remove!(reg)
    block.result = samples
    reg
end

apply! (generic function with 18 methods)

In [27]:
m = Measure()
mr = MeasureAndRemove()

Total: Yao.Blocks.AnySize, DataType: Bool
MeasureAndRemove


In [30]:
reg2 = copy(reg)
pre = nothing
for i in 1:5
    reg2 = apply!(reg2, m)
    @test reg2 |> isnormalized
    if pre!=nothing
        @test pre == m.result
    end
    pre = m.result
end

In [29]:
reg2 = focus!(copy(reg), 2:3)
reg2 = apply!(reg2, mr)
@test nqubits(reg2) == 3
@test nactive(reg2) == 0
@test reg2 |> isnormalized

Test Passed


In [23]:
reg

DefaultRegister{3, Complex{Float64}}
    active qubits: 5/5